In [2]:
import censusdata
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
import geopandas as gpd

In [3]:
#censusdata.search('acs5', 2015, 'concept', 'hispanic')

In [15]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B03002'))

In [12]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B03002_001E', 'B03002_002E', 'B03002_012E'])

df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B03002_001E' : 'Total', 'B03002_002E' : 'Non-Hispanic/Latino', 
                     'B03002_012E' : 'Hispanic/Latino'}, inplace = True)

for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df

,TRACT,Total,Non-Hispanic/Latino,Hispanic/Latino
0,802,6701,5613,1088
1,10600,3168,3001,167
2,40100,2442,2358,84
3,40300,4665,4152,513
4,50400,2284,1136,1148
...,...,...,...,...
199,82100,5998,4141,1857
200,60200,2103,2054,49
201,90400,4396,2566,1830
202,120500,2850,1654,1196


In [13]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_hisp = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_hisp.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_hisp)):
    prct = prct_hisp['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_hisp[c].iloc[i] += round(df[filt][c] * dec)

In [14]:
prct_hisp.to_csv('Precinct_Hispanic_Population_2014-2019.csv')

In [32]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_hisp_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_hisp.columns[1:])
neigh_hisp_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_hisp_11_15)):
    neighb = neigh_hisp_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_hisp['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_hisp.columns[1:]:
                try:
                    neigh_hisp_11_15[c].iloc[i] += round(prct_hisp[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_hisp_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_hisp_11_15[c] = neigh_hisp_11_15[c].round().astype(int)

repl = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2011-15.csv') 
neigh_hisp_11_15['Total'] = repl['TOTAL']
        
cols = list(neigh_hisp_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_hisp_11_15)):
        neigh_hisp_11_15.loc[i, c] = "{:.0%}".format(neigh_hisp_11_15.loc[i, c]/neigh_hisp_11_15.loc[i, 'Total'])
 
neigh_hisp_11_15

,NEIGHBORHOOD,Total,Non-Hispanic/Latino,Hispanic/Latino
0,Roslindale,29024,78%,22%
1,Jamaica Plain,42759,74%,26%
2,Mission Hill,13920,82%,18%
3,Longwood,6241,88%,12%
4,Bay Village,1399,94%,6%
5,Leather District,537,94%,6%
6,Chinatown,2718,94%,6%
7,North End,6964,96%,4%
8,Roxbury,50601,72%,28%
9,South End,23667,87%,13%


In [33]:
neigh_hisp_11_15.to_csv('Boston_Neighborhood_Estimated_Hispanic_Demographics_ACS_2011-2015.csv')

In [23]:
df2 = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B03002_001E', 'B03002_002E', 'B03002_012E'])

df2 = df2.reset_index(drop=False)
df2.rename(columns = {'index' : 'TRACT', 'B03002_001E' : 'Total', 'B03002_002E' : 'Non-Hispanic/Latino', 
                     'B03002_012E' : 'Hispanic/Latino'}, inplace = True)

for x in range(len(df)):
    df2.loc[x, 'TRACT'] = df2['TRACT'].unique()[x].geo[2][1]
    
df2 = df2.astype({"TRACT": int})

df2

,TRACT,Total,Non-Hispanic/Latino,Hispanic/Latino
0,802,6701,5613,1088
1,10600,3168,3001,167
2,40100,2442,2358,84
3,40300,4665,4152,513
4,50400,2284,1136,1148
...,...,...,...,...
199,82100,5998,4141,1857
200,60200,2103,2054,49
201,90400,4396,2566,1830
202,120500,2850,1654,1196


In [24]:
prct_hisp2 = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df2.columns[1:])

prct_hisp2.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_hisp2)):
    prct = prct_hisp2['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df2['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_hisp2[c].iloc[i] += round(df2[filt][c] * dec)

In [34]:
neigh_hisp_15_19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_hisp2.columns[1:])
neigh_hisp_15_19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_hisp_15_19)):
    neighb = neigh_hisp_15_19['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_hisp2['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_hisp2.columns[1:]:
                try:
                    neigh_hisp_15_19[c].iloc[i] += round(prct_hisp2[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_hisp_15_19.columns:
    if c != 'NEIGHBORHOOD':
        neigh_hisp_15_19[c] = neigh_hisp_15_19[c].round().astype(int)

repl = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2015-19.csv') 
neigh_hisp_15_19['Total'] = repl['TOTAL']
        
cols = list(neigh_hisp_15_19.columns)[2:]

for c in cols:
    for i in range(len(neigh_hisp_15_19)):
        neigh_hisp_15_19.loc[i, c] = "{:.0%}".format(neigh_hisp_15_19.loc[i, c]/neigh_hisp_15_19.loc[i, 'Total'])

neigh_hisp_15_19

,NEIGHBORHOOD,Total,Non-Hispanic/Latino,Hispanic/Latino
0,Roslindale,30207,77%,23%
1,Jamaica Plain,43871,77%,23%
2,Mission Hill,14251,80%,20%
3,Longwood,6549,87%,13%
4,Bay Village,1465,94%,6%
5,Leather District,621,94%,6%
6,Chinatown,3113,94%,6%
7,North End,6785,93%,7%
8,Roxbury,53772,70%,29%
9,South End,24202,85%,15%


In [35]:
neigh_hisp_15_19.to_csv('Boston_Neighborhood_Estimated_Hispanic_Demographics_ACS_2015-2019.csv')